In [ ]:
import pandas as pd
import numpy as np
import mlflow
import json
import re
import uuid

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold


In [ ]:
# 데이터 전처리 코드 기존 데이터를 읽어와 전처리된 csv 파일로 저장(또는 곧바로 사용 가능)
# 데이터 읽어오기 및 전처리 함수
def parse_json(item):
    try:
        data = json.loads(item)  # JSON 문자열을 파싱
        if isinstance(data, dict):  # 단일 딕셔너리라면 리스트로 감싸기
            return [data]
        return data  # 리스트라면 그대로 반환
    except json.JSONDecodeError:
        return None  # JSON 파싱 오류 발생 시 None 반환

def json_column_to_column(df):
    df['parsed_data'] = df['contents'].apply(parse_json)
    df_exploded = df.explode('parsed_data', ignore_index=True)
    return pd.concat([df_exploded.drop(columns=['contents', 'parsed_data']), pd.json_normalize(df_exploded['parsed_data'])], axis=1)

# 데이터 읽어오기
df = pd.read_csv('./take_hotel.csv')
convert_df = json_column_to_column(df)
convert_df = convert_df[['room_name', 'reg_date', 'itemName']]

# 키워드 리스트로 필터링
keywords = [
    "Chicken Karaage & Fried  Dishes (치킨 가라아게 & 모둠 튀김)", "Beef Truffle Zappaghetti(소고기 트러플 짜파게티)",
    "Mini Burger Set (미니버거 세트) 음료 선택 필수", "Bluemoon Draft Beer (블루문 생맥주)", "Miller Draft Beer (밀러 생맥주)",
    "Pig Hocks Set (순살 족발 세트)", "Beef Brisket Kimchi Pilaf(차돌 깍두기 볶음밥)", "Zero Coke (제로 콜라)",
    "Margherita Pizza(마르게리따 피자)", "Shine Musket Ade (샤인머스켓 에이드)", "Hamburg Steak (함박 스테이크)",
    "Truffle mushrooms Pizza(트러플 버섯 피자)", "Tteokbokki(떡볶이)", "Truffle Creamy Gnocchi Pasta(트러플 크림 뇨끼)",
    "Americano (아메리카노) 요청사항에 HOT/ICE 부탁드립니다.", "Lemon Ade (레몬 에이드)", "Coke (코카콜라)",
    "Seafood Oil Casareccia Pasta (씨푸드 오일 파스타)", "Prosciutto Rucola Pizza (프로슈토 루꼴라 피자)", 
    "Abalone Porridge + Side  Dishes (전복죽 + 반찬)", "Starlight Chungha (별빛 청하)", "LA Galbi Set (LA 갈비 세트)", 
    "Zero Sprite(제로 스프라이트)", "Grilled Camembert Cheese(까망베르 치즈 구이)", "Grapefruit Ade (자몽 에이드)",
    "Pancake Plate(팬케이크 플레이트)", "Sprite (스프라이트)", "Chicken Burger(치킨버거)", "Margherita Pizza(마르게리따타피자)",
    "Prosciutto Rucola Pizza(프로슈토 루꼴라 피자)", "Americano (아메리카노)", "Hwayo25 Soju (화요 25)", 
    "Mini Burger set", "Korea Traditional Pancakes (모둠 전)", "ILPUM Jinro Soju (일품 진로 소주)",
    "Abalone Porridge  (전복죽)", "Mini Burger Set (미니버거 세트)", "Creamy Pasta (크림 파스타)", 
    "Melon prosciutto(메론 프로슈토)", "Shine musket(샤인머스켓 에이드)", "Abalone Porridge + side (전복죽 + 반찬)",
    "Chicken Karaage & Fries Plate(치킨 가라아게 & 모둠튀김)", "Seafood Oil Casareccia Pasta(씨푸드 오일 까사레치아 파스타)", 
    "miller Draft beer  (밀러 맥주)", "Pig Hocks set(순살 족발세트)", "Beer (캘리)", "Coca cola(콜라)", 
    "Melon Prosciutto (메론 프로슈토)", "Seafood Oil Casareccia Pasta(씨푸트 오일 까사레치아 파스타)", 
    "LA Galbi set(LA 갈비 세트)", "bluemoon Draft beer  (블루문 맥주)", "Hamburg Steak(함박 스테이크)", 
    "Sprite(스프라이트)", "Soju (새로)", "Scones(스콘)", "Coca cola(코카콜라)", "Sprite(스프라이트, 제로 택 1)",
    "Korea Traditional Pancake(모둠전)", "ILpum Jinro Soju (일품 진로 소주)", "Zero cola (제로 콜라)", 
    "Chicken Karaage & Fried  Plate", "Seafood Oil Casareccia Pasta", "Starlight Chung Ha(달빛 청하)", 
    "아메리카노", "Hwayo25 soju(화요 25 소주)", "Grapefruit Ade(자몽 에이드)", "Draft beer(bluemoon, 생맥주)", 
    "Prosciutto Rucola Pizza", "LA Galbi set", "Pig Hocks set (순살 족발 세트)", "Draft beer(bluemoon,  miller 택 1)"
]


escaped_keywords = [re.escape(keyword) for keyword in keywords]
pattern = '|'.join(escaped_keywords)  # 이스케이프된 키워드를 '|'로 연결

filtered_df = convert_df[convert_df['itemName'].str.contains(pattern, case=False, na=False)]

# 키워드 매핑
keyword_dict = {
    # Chicken and Fried Dishes
    "Chicken Karaage & Fried  Dishes (치킨 가라아게 & 모둠 튀김)": "Chicken Karaage & Fried Dishes",
    "Chicken Karaage & Fries Plate(치킨 가라아게 & 모둠튀김)": "Chicken Karaage & Fried Dishes",
    "Chicken Karaage & Fried  Plate": "Chicken Karaage & Fried Dishes",

    # Beef and Other Main Dishes
    "Beef Truffle Zappaghetti(소고기 트러플 짜파게티)": "Beef Truffle Zappaghetti",
    "Beef Brisket Kimchi Pilaf(차돌 깍두기 볶음밥)": "Beef Brisket Kimchi Pilaf",
    "LA Galbi Set (LA 갈비 세트)": "LA Galbi Set",
    "LA Galbi set(LA 갈비 세트)": "LA Galbi Set",
    "LA Galbi set": "LA Galbi Set",
    
    # Pig Hocks
    "Pig Hocks Set (순살 족발 세트)": "Pig Hocks Set",
    "Pig Hocks set(순살 족발세트)": "Pig Hocks Set",
    "Pig Hocks set (순살 족발 세트)": "Pig Hocks Set",
    
    # Burger
    "Mini Burger Set (미니버거 세트) 음료 선택 필수": "Mini Burger Set",
    "Mini Burger set": "Mini Burger Set",
    "Mini Burger Set (미니버거 세트)": "Mini Burger Set",
    "Chicken Burger(치킨버거)": "Chicken Burger",
    
    # Pizza
    "Margherita Pizza(마르게리따 피자)": "Margherita Pizza",
    "Margherita Pizza(마르게리따타피자)": "Margherita Pizza",
    "Prosciutto Rucola Pizza (프로슈토 루꼴라 피자)": "Prosciutto Rucola Pizza",
    "Prosciutto Rucola Pizza(프로슈토 루꼴라 피자)": "Prosciutto Rucola Pizza",
    "Prosciutto Rucola Pizza": "Prosciutto Rucola Pizza",
    "Truffle mushrooms Pizza(트러플 버섯 피자)": "Truffle Mushrooms Pizza",

    # Pasta and Gnocchi
    "Seafood Oil Casareccia Pasta (씨푸드 오일 파스타)": "Seafood Oil Casareccia Pasta",
    "Seafood Oil Casareccia Pasta(씨푸드 오일 까사레치아 파스타)": "Seafood Oil Casareccia Pasta",
    "Seafood Oil Casareccia Pasta(씨푸트 오일 까사레치아 파스타)": "Seafood Oil Casareccia Pasta",
    "Seafood Oil Casareccia Pasta": "Seafood Oil Casareccia Pasta",
    "Truffle Creamy Gnocchi Pasta(트러플 크림 뇨끼)": "Creamy Pasta",
    "Creamy Pasta (크림 파스타)": "Creamy Pasta",

    # Porridge
    "Abalone Porridge + Side  Dishes (전복죽 + 반찬)": "Abalone Porridge",
    "Abalone Porridge + side (전복죽 + 반찬)": "Abalone Porridge",
    "Abalone Porridge  (전복죽)": "Abalone Porridge",

    # Pancakes
    "Pancake Plate(팬케이크 플레이트)": "Korean Traditional Pancakes",
    "Korea Traditional Pancakes (모둠 전)": "Korean Traditional Pancakes",
    "Korea Traditional Pancake(모둠전)": "Korean Traditional Pancakes",

    # Others
    "Melon prosciutto(메론 프로슈토)": "Melon Prosciutto",
    "Melon Prosciutto (메론 프로슈토)": "Melon Prosciutto",
    "Hamburg Steak (함박 스테이크)": "Hamburg Steak",
    "Hamburg Steak(함박 스테이크)": "Hamburg Steak",
    "Tteokbokki(떡볶이)": "Tteokbokki",
    "Grilled Camembert Cheese(까망베르 치즈 구이)": "Grilled Camembert Cheese",
    "Scones(스콘)": "Scones",

    # Soju
    "Hwayo25 Soju (화요 25)": "Soju",
    "Hwayo25 soju(화요 25 소주)": "Soju",
    "ILPUM Jinro Soju (일품 진로 소주)": "Soju",
    "ILpum Jinro Soju (일품 진로 소주)": "Soju",
    "Soju (새로)": "Soju",
    "Starlight Chungha (별빛 청하)": "Soju",
    "Starlight Chung Ha(달빛 청하)": "Soju",

    # Beer
    "Bluemoon Draft Beer (블루문 생맥주)": "Beer",
    "Draft beer(bluemoon, 생맥주)": "Beer",
    "Draft beer(bluemoon,  miller 택 1)": "Beer",
    "bluemoon Draft beer  (블루문 맥주)": "Beer",
    "Miller Draft Beer (밀러 생맥주)": "Beer",
    "miller Draft beer  (밀러 맥주)": "Beer",
    "Beer (캘리)": "Beer",
    "Kelly Beer": "Beer",

    # 탄산음료
    "Coke (코카콜라)": "탄산음료",
    "Coca cola(콜라)": "탄산음료",
    "Coca cola(코카콜라)": "탄산음료",
    "Zero Coke (제로 콜라)": "탄산음료",
    "Zero cola (제로 콜라)": "탄산음료",
    "Sprite (스프라이트)": "탄산음료",
    "Sprite(스프라이트)": "탄산음료",
    "Sprite(스프라이트, 제로 택 1)": "탄산음료",
    "Zero Sprite(제로 스프라이트)": "탄산음료",

    # 에이드류
    "Lemon Ade (레몬 에이드)": "에이드",
    "Grapefruit Ade (자몽 에이드)": "에이드",
    "Grapefruit Ade(자몽 에이드)": "에이드",
    "Shine Musket Ade (샤인머스켓 에이드)": "에이드",
    "Shine musket(샤인머스켓 에이드)": "에이드",

    # Coffee and Tea
    "Americano (아메리카노) 요청사항에 HOT/ICE 부탁드립니다.": "Americano",
    "Americano (아메리카노)": "Americano",
    "아메리카노": "Americano"
}


filtered_df.loc[:, 'itemName'] = filtered_df['itemName'].replace(keyword_dict)
filtered_df

In [ ]:
import requests
import json
from pprint import pprint

url = "http://<publicIP>:5001/invocations"
headers = {"Content-Type": "application/json"}

data = {
    "dataframe_split": {
        "columns": ["room_name"],
        "data": [2606,1906]
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))
pprint(response.json())

In [ ]:
# csv파일저장
filtered_df.to_csv("filtered_take_hotel.csv",index=False)

In [ ]:
# csv파일읽기
filtered_df = pd.read_csv("filtered_take_hotel.csv")
filtered_df

In [ ]:
# train / test split
test_percent = 0.2
total_len = len(filtered_df)
train_len = int(total_len*(1-test_percent))

train_df = filtered_df[:train_len].copy()
test_df = filtered_df[train_len:].copy()

In [ ]:
train_df.to_csv("train_original.csv",index=False)
test_df.to_csv("test_original.csv",index=False)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df['itemName'].value_counts()

In [ ]:
test_df['itemName'].value_counts()

In [ ]:
# 기본적으로 주문이 많이 된 아이템 목록 추출
best_item = train_df['itemName'].value_counts().index.tolist()
print(best_item)

In [ ]:
# Stratified K-Fold 적용
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1212)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df,train_df['room_name'])):
    print(f"Fold {fold + 1}")

    train_data = train_df.iloc[train_idx].copy()
    valid_data = train_df.iloc[valid_idx].copy()

    train_data.to_csv(f"train_data_{fold}.csv", index=False)
    valid_data.to_csv(f"valid_data_{fold}.csv", index=False)

In [ ]:
##### 한개의 train_data를 이용하여 latent_cf 를 생성하고 추론 결과를 반환해보기
# 주문 횟수를 평점행렬로 만들어주기
# train_df["count"]=1

# 주문 횟수를 평점행렬로 만들어주기
# ratings_matrix = pd.pivot_table(train_df,values='count',index='room_name',columns='itemName',aggfunc='sum',fill_value=0)
# ratings_matrix


In [ ]:
##### 한개의 train_data를 이용하여 latent_cf 를 생성하고 추론 결과를 반환해보기
# 주문 횟수를 평점행렬로 만들어주기
ratings_matrix = pd.pivot_table(train_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)
ratings_matrix

In [ ]:
def get_rmse(R, P ,Q):
    pred_matrix = np.dot(P,Q.T)

    R_not_nans = R[~np.isnan(R)]

    pred_matrix_not_nanas = pred_matrix[~np.isnan(R)]

    rmse = np.sqrt(mean_squared_error(R_not_nans,pred_matrix_not_nanas))

    return rmse

In [ ]:
def train_latent_cf(train_matrix,k_factor,learning_rate,r_lambda):
    R = train_matrix
    
    num_rooms, num_items = R.shape

    K = k_factor

    P = np.random.normal(scale=1./K, size=(num_rooms,K))
    Q = np.random.normal(scale=1./K, size=(num_items,K))

    epochs = 1000

    not_nans = [ (i,j,R[i,j]) for i in range(num_rooms) for j in range(num_items) if R[i,j]>0]

    not_nans_index = np.argwhere(R > 0)  # 조건을 만족하는 (i, j) 인덱스 반환
    not_nans_values = R[R > 0]              # 조건을 만족하는 값 반환
    not_nans = [(i, j, val) for (i, j), val in zip(not_nans_index, not_nans_values)]
    
    for epoch in range(epochs):
        for i,j,r in not_nans:
            error_i_j = r - np.dot(P[i,:],Q[j,:].T)
            
            P[i,:] = P[i,:] + learning_rate *(error_i_j*Q[j,:]-r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate *(error_i_j*P[i,:]-r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q)
        if (epoch%100) == 0:
            print("iteration step :",epoch," rmse :",rmse)

    return np.dot(P,Q.T)

In [ ]:
class LatentFactorCF(mlflow.pyfunc.PythonModel):
    def __init__(self,pred_df,basic_recommend_item):
        self.pred_df = pred_df
        self.basic_recommend_item = basic_recommend_item
        
    def predict(self, context, model_input, params=None):
        answer = {}
        for room in model_input['room_name']:
            if room in self.pred_df.index:
                target_row = self.pred_df.loc[room]
                sorted_items = target_row.sort_values(ascending=False)
                sorted_item_names = sorted_items.index.tolist()
                answer[room]=sorted_item_names
            else:
                answer[room]=self.basic_recommend_item
        return answer

In [ ]:
def calculate_best_item_accuracy(valid_pivot_table, best_items):
    # 모든 호실의 추천된 아이템 구매 횟수 총합
    total_recommended_purchases = 0

    # 전체 구매 횟수
    total_purchases = valid_pivot_table.sum().sum()

    for room_id in valid_pivot_table.index:
        # 상위 5개의 추천 아이템 중 실제 pivot_table에 존재하는 아이템만 필터링
        existing_items = [item for item in best_items[:3] if item in valid_pivot_table.columns]

        if not existing_items:  # 만약 존재하는 아이템이 없다면
            continue  # 해당 room_id는 스킵

        # 추천된 아이템의 구매 횟수 합산
        total_recommended_purchases += valid_pivot_table.loc[room_id, existing_items].sum()

    # 정확도 계산
    accuracy = total_recommended_purchases / total_purchases if total_purchases > 0 else 0
    return accuracy

In [ ]:
def calculate_overall_accuracy(valid_pivot_table, recommendations):
    # 모든 호실의 추천된 아이템 구매 횟수 총합
    total_recommended_purchases = 0
    
    # 전체 구매 횟수
    total_purchases = valid_pivot_table.sum().sum()

    for room_id, recommended_items in recommendations.items():
        # 추천된 아이템의 구매 횟수 합산
        # 추천된 아이템 중 실제 valid_pivot_table에 존재하는 아이템만 필터링
        existing_items = [item for item in recommended_items[:3] if item in valid_pivot_table.columns]

        if not existing_items:  # 만약 존재하는 아이템이 없다면
            continue  # 해당 room_id는 스킵

        # 추천된 아이템의 구매 횟수 합산
        total_recommended_purchases += valid_pivot_table.loc[room_id, existing_items].sum()
    
    # 정확도 계산
    accuracy = total_recommended_purchases / total_purchases if total_purchases > 0 else 0
    return accuracy

In [ ]:
mlflow.set_tracking_uri("http://<PublicIP>:5000")
mlflow.set_experiment("hotel_item")

UNIQUE_PREFIX = str(uuid.uuid4())[:8]

# optuna를 통한 HPO 진행
def objective(trial):
    trial.suggest_int("k_factor",64,1024,step=64)
    trial.suggest_float("learning_rate", 1e-4, 5e-2)
    trial.suggest_float("r_lambda", 1e-4,5e-2)

    # n_epoch 동안 모델 학습  
    
    # 모델 학습을 진행하고 해당 결과를 run_name으로 기록하기
    run_name = f"{UNIQUE_PREFIX}-{trial.number}"
    
    with mlflow.start_run(run_name=run_name):
        mlflow.log_params(trial.params)

        latent_accuracy_list = []
        simple_accuracy_list = []
        for fold_idx in range(4):   # 4-Fold
        # 주문 횟수를 평점행렬로 만들어주기
            train_df = pd.read_csv(f"train_data_{fold_idx}.csv")
            valid_df = pd.read_csv(f"valid_data_{fold_idx}.csv")

            # 주문 횟수를 평점행렬로 만들어주기
            ratings_matrix = pd.pivot_table(train_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

            pred_R = train_latent_cf(ratings_matrix.to_numpy(),k_factor=trial.params["k_factor"],learning_rate=trial.params['learning_rate'],r_lambda=trial.params['r_lambda'])

            pred_df = pd.DataFrame(pred_R,index=ratings_matrix.index,columns=ratings_matrix.columns)
            model = LatentFactorCF(pred_df,best_item)
            
            valid_predict_dict = model.predict(None,valid_df)

            vaild_answer_pivot_table = pd.pivot_table(valid_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)
            
            valid_accuracy = calculate_overall_accuracy(vaild_answer_pivot_table, valid_predict_dict)
            
            
            accuracy = calculate_best_item_accuracy(vaild_answer_pivot_table, best_item)
            
            latent_accuracy_list.append(valid_accuracy)
            simple_accuracy_list.append(accuracy)
            
        mlflow.log_metric("latentcf_recommend_accuracy", np.mean(latent_accuracy_list))
        mlflow.log_metric("simple_recommend_accuracy", np.mean(simple_accuracy_list))

    return np.mean(latent_accuracy_list)


In [ ]:
import optuna

sampler = optuna.samplers.RandomSampler(seed=2024)
study = optuna.create_study(sampler=sampler, study_name="latent_cf", direction="maximize")

# optimize
study.optimize(objective, n_trials=100)


In [ ]:
best_params = study.best_params
print(best_params)

In [ ]:
import mlflow

mlflow.set_tracking_uri("http://<PublicIP>:5000")
mlflow.set_experiment("hotel_item")


# best_params 기준으로 전체 Train 데이터에 대하여 재학습 진행
best_params = study.best_params

# train/test전체 데이터 가져오기 
train_df = pd.read_csv("train_original.csv")
test_df = pd.read_csv("test_original.csv")

# 주문 횟수를 평점행렬로 만들어주기
ratings_matrix = pd.pivot_table(train_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

pred_R = train_latent_cf(ratings_matrix.to_numpy(),k_factor=best_params["k_factor"],learning_rate=best_params['learning_rate'],r_lambda=best_params['r_lambda'])

pred_df = pd.DataFrame(pred_R,index=ratings_matrix.index,columns=ratings_matrix.columns)
best_item = ['Beer', 'Chicken Karaage & Fried Dishes', 'Mini Burger Set', '탄산음료', 'Beef Truffle Zappaghetti', '에이드', 'Pig Hocks Set', 'Margherita Pizza', 'Beef Brisket Kimchi Pilaf', 'Soju', 'Americano', 'Prosciutto Rucola Pizza', 'Hamburg Steak', 'Seafood Oil Casareccia Pasta', 'Abalone Porridge', 'Truffle Mushrooms Pizza', 'Tteokbokki', 'Creamy Pasta', 'LA Galbi Set', 'Korean Traditional Pancakes', 'Grilled Camembert Cheese', 'Chicken Burger', 'Melon Prosciutto', 'Scones']

model = LatentFactorCF(pred_df,best_item) 

# 학습된 모델로 test 데이터에 추론 결과 살펴보기
test_predict_dict = model.predict(None,test_df)
test_answer_pivot_table = pd.pivot_table(test_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

test_accuracy = calculate_overall_accuracy(test_answer_pivot_table, test_predict_dict)
accuracy = calculate_best_item_accuracy(test_answer_pivot_table, best_item)
print("모델 정확도 :",test_accuracy)
print("단순추천 정확도 :",accuracy)


In [ ]:
# 모델 입력 예제를 DataFrame으로 정의
input_example = pd.DataFrame({
    "room_name": [901, 902, 2704],
})

with mlflow.start_run(run_name="Best_Model_Run") as run:
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=model,
        input_example=input_example,
    )

In [ ]:
# 모델 registry에 등록
model_uri = f"runs:/{run.info.run_id}/model"
registered_model = mlflow.register_model(model_uri=model_uri, name="latent_best_model") 

In [ ]:
import requests
import json

url = "http://<PublicIP>:5001/invocations"
headers = {"Content-Type": "application/json"}

data = {
    "dataframe_split": {
        "columns": ["room_name"],
        "data": [2606,1906]
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

In [ ]:
import requests
import json

test_df = pd.read_csv("test_original.csv")


url = "http://<publicIP>:5001/invocations"
headers = {"Content-Type": "application/json"}

data = {
    "dataframe_split": test_df.to_dict(orient="split")
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

# 학습된 모델로 test 데이터에 추론 결과 살펴보기
# test_predict_dict = model.predict(None,test_df)
test_predict_dict = requests.post(url, headers=headers, data=json.dumps(data)).json()['predictions']
test_predict_dict = {int(k): v for k, v in test_predict_dict.items()}
test_answer_pivot_table = pd.pivot_table(test_df,index='room_name',columns='itemName',aggfunc='size',fill_value=0)

test_accuracy = calculate_overall_accuracy(test_answer_pivot_table, test_predict_dict)
accuracy = calculate_best_item_accuracy(test_answer_pivot_table, best_item)
print("모델 정확도 :",test_accuracy)
print("단순추천 정확도 :",accuracy)